In [1]:
import torch
if torch.cuda.is_available():
    print(f"Версия PyTorch: {torch.__version__}")
    print(f"Версия CUDA (если установлена): {torch.version.cuda}")
    print(f"Версия cuDNN: {torch.backends.cudnn.version()}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
else:
    device_str = 'cpu'
    print("CUDA not available. Using device: cpu")

Версия PyTorch: 2.7.1+cu118
Версия CUDA (если установлена): 11.8
Версия cuDNN: 90100
Number of GPUs available: 2


In [2]:
import xformers

In [3]:
xformers.__version__

'0.0.31'

In [4]:
import lightning

In [5]:
lightning.__version__

'2.5.2'

In [6]:
# Copyright (c) Facebook, Inc. and its affiliates. All rights reserved.
#
# This source code is licensed under the BSD license found in the
# LICENSE file in the root directory of this source tree.

# A MinGPT + Lightning + xFormers example Code from Sean Naren (@seannaren)
# This is an hommage to https://github.com/karpathy/minGPT

import math
import os

import pytorch_lightning as pl
import torch
import torch.nn as nn
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.utilities import rank_zero_info
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, RandomSampler


In [7]:
!poetry run python -m xformers.info

xFormers 0.0.31
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF-pt:            available
memory_efficient_attention.cutlassB-pt:            available
memory_efficient_attention.fa2F@2.5.7-pt:          available
memory_efficient_attention.fa2B@2.5.7-pt:          available
memory_efficient_attention.fa3F@0.0.0:             unavailable
memory_efficient_attention.fa3B@0.0.0:             unavailable
memory_efficient_attention.fa3F_splitKV@0.0.0:     unavailable
memory_efficient_attention.triton_splitKF:         available
indexing.scaled_index_addF:                        available
indexing.scaled_index_addB:                        available
indexing.index_select:                             available
sp24.sparse24_sparsify_both_ways:                  avai

In [10]:
from xformers.factory import xFormer, xFormerConfig, xFormerEncoderConfig

ModuleNotFoundError: No module named 'xformers.factory'

In [ ]:
import torch
from torch import nn

class MolecularEmbedder(nn.Module):
    def __init__(self, vocab_size, d_model=256, n_layers=6, n_heads=8):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        
        encoder_layer_template = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            batch_first=True 
        )
        
        self.encoder = nn.TransformerEncoder(  # <--- xformer in env
            encoder_layer=encoder_layer_template,
            num_layers=n_layers
        )

        self.predictor = nn.Linear(d_model, vocab_size)